In [2]:
import urllib

import shutil
import urllib.request as request
from contextlib import closing

import os
import io
import gzip
import tarfile

import xml.etree.ElementTree as ET

import re

import json

from datetime import datetime


###############################################################
### EXCEPTIONS

class GeoError(Exception):
    """Base class for GEO Errors"""
    pass

class GeoRetrivalError(GeoError):
    """Problem in retriving data from GEO """
    pass

class GeoParseError(GeoError):
    """Problem in parsing data from GEO """
    pass

###############################################################


##############################################################
### HELPER FUNCTIONS

def sanitize_sra(s):
    """
    To avoid redundancy and ambiguity, we search for SRR ids
    surrounded by > and <. So we remove those boundary charactgers.

    >SRR12345< -> SRR12345
    """
    return s.replace("<", "").replace(">", "")



def extract_srr_from_str(srr_page_text):
    """
    extracts the SRR accession numbers
    from the xml string of the SRA page.

    Returns a list of SRR accession IDs.
    """

    re_pattern   = re.compile(r">SRR[0-9]+<", re.DOTALL)
    raw_finds    = re_pattern.findall(srr_page_text)
    actual_finds = list( map( sanitize_sra , raw_finds ) )

    return actual_finds



def get_srr_accessions(sra_url):
    """
    Returns a list of SRR accession ids from the sra url
    Example URL: https://www.ncbi.nlm.nih.gov/sra?term=SRX9173299
    """
    page_text = request.urlopen(sra_url).read().decode("utf-8")
    return extract_srr_from_str(page_text)



def sanitize_text(t):
    """
    Clears all white space from the text
    We keep this as a separate function in case
    text needs further sanitizing
    """
    return t.strip()



def parse_sample(sample_node):
    """
    Reads experiment info from the XML tree
    Returns a dictionary containing experiment metadata
    The dictionary also contains
    SRA accession ids of the sequencing data
    """
    sample_dict = {}

    for x in sample_node:
        if x.tag.lower().endswith("title"):
            sample_dict["title"] =  sanitize_text( x.text )
        if x.tag.lower().endswith("accession"):
            sample_dict["accession"] =  sanitize_text( x.text )
        if x.tag.lower().endswith("channel"):
            for c in x:
                if c.tag.lower().endswith("organism"):
                    sample_dict["organism"] =  sanitize_text( c.text )
                if c.tag.lower().endswith("characteristics"):
                    if c.attrib.get("tag", "").lower() == "cell line":
                        sample_dict["cell_line"] =  sanitize_text( c.text )
        if x.tag.lower().endswith("status"):
            for s in x:
                if s.tag.lower().endswith("submission-date"):
                    sample_dict["submission-date"] =  sanitize_text( s.text )

        if x.tag.lower().endswith("relation"):
            if x.attrib.get("type", "").lower() == "sra":
                sample_dict["sra_link"] = x.attrib["target"]
                sample_dict["srr_ids"]  = get_srr_accessions( x.attrib["target"] )

    return sample_dict



def parse_study_string(xml_string):
    """
    parses the xml string and returns a dictionary
    containing the metadata of the study and its experiments
    """

    study_root     = ET.fromstring(xml_string)
    geo_study_dict = { "experiments" : dict() }

    for a in study_root:
        if a.tag.lower().endswith("series"):
            for s in a:
                if s.tag.lower().endswith("accession"):
                    geo_study_dict["accession"] = s.text
                if s.tag.lower().endswith("title"):
                    geo_study_dict["title"] = s.text
                if s.tag.lower().endswith("summary"):
                    geo_study_dict["abstract"] = s.text
                if s.tag.lower().endswith("type"):
                    geo_study_dict["type"] = s.text
                if s.tag.lower().endswith("overall-design"):
                    geo_study_dict["overall-design"] = s.text

                if s.tag.lower().endswith("status"):
                    for t in s:
                        if t.tag.lower().endswith("submission-date"):
                            geo_study_dict["submission-date"] = t.text

        if a.tag.lower().endswith("sample"):
            geo_study_dict["experiments"][a.attrib["iid"]] = parse_sample(a)

    return geo_study_dict



def read_miniml_from_GEO(file_ftp_path):
    """
    returns the contents of the xml file as a string

    By default, the file is in tgz = tar.gz format.
    So the file needs to be unzipped and untarred
    There should be a file inside the tar file with the same name except for the tgz
    For example, if the path is
    "ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE158nnn/GSE158374/miniml/GSE158374_family.xml.tgz"
    then there should be the file
    "GSE158374_family.xml" in the tar.gz file.
    We extract that file and read its contetns to a string
    This string is return
    """

    # We assume the filename ends with .tgz
    # so we leave it out
    name_pieces   = file_ftp_path.split("/")[-1].split(".")
    xml_file_name = ".".join(name_pieces[:-1])


    with closing(request.urlopen(file_ftp_path)) as ftp_stream:
        mybuffer     = io.BytesIO(ftp_stream.read())
        mytar        = tarfile.open(fileobj=mybuffer,mode =  'r')
        xml_contents = mytar.extractfile(xml_file_name)
        contents     = xml_contents.read().decode("utf-8")

        return contents



def get_study_metadata_from_miniml(file_ftp_url):
    """
    Extracts the metadata of the study from the given ftp miniml file path.

    Example path: ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE158nnn/GSE158374/miniml/GSE158374_family.xml.tgz
    """

    miniml_file_contents = read_miniml_from_GEO(file_ftp_url)
    study_metadata_dict  = parse_study_string(miniml_file_contents)

    return study_metadata_dict


def get_html_page_of_study(study_accession):

    study_geo_url = "https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={}".format( study_accession )

    try:
        with closing(request.urlopen(study_geo_url)) as http_stream:
            html_code = http_stream.read().decode("utf-8")
    except URLError:
        return ""

    return html_code


def get_miniml_url(html_code, study_accession):
    search_pattern  = re.compile("ftp://ftp.ncbi.nlm.nih.gov/geo/series/[A-Z]*[0-9]*[n]*/[A-Z]*[0-9]*/miniml/")

    search_result   = search_pattern.findall(html_code)

    if len(search_result) == 0:
        return ""

    miniml_folder   = search_result[0]
    m               = re.match(r".*/(?P<study_id>[A-Z]+[0-9]+)/.*", miniml_folder)
    obtained_acc    = m.group("study_id")

    assert study_accession == obtained_acc

    miniml_url      = "{}{}_family.xml.tgz"\
                      .format(miniml_folder, study_accession)

    return miniml_url


def get_study_dict_from_geo_id(study_accession):
    html_code = get_html_page_of_study(study_accession)

    if not html_code:
        print("Couldn't find the study {}".format(study_accession))
        return None

    study_miniml_url = get_miniml_url(html_code, study_accession)

    if not study_miniml_url:
        print("Couldn't find the study {}".format(study_accession))
        return None

    study_dict = get_study_metadata_from_miniml(study_miniml_url)

    return study_dict

In [4]:
study_dict = get_study_dict_from_geo_id("GSE150316")

In [5]:
study_dict

{'experiments': {'GSM4546576': {'submission-date': '2020-05-11',
   'title': 'case1-lung1',
   'accession': 'GSM4546576',
   'organism': 'Homo sapiens',
   'sra_link': 'https://www.ncbi.nlm.nih.gov/sra?term=SRX8325539',
   'srr_ids': ['SRR11772358']},
  'GSM4546577': {'submission-date': '2020-05-11',
   'title': 'case1-lung2',
   'accession': 'GSM4546577',
   'organism': 'Homo sapiens',
   'sra_link': 'https://www.ncbi.nlm.nih.gov/sra?term=SRX8325540',
   'srr_ids': ['SRR11772359']},
  'GSM4546578': {'submission-date': '2020-05-11',
   'title': 'case1-lung3',
   'accession': 'GSM4546578',
   'organism': 'Homo sapiens',
   'sra_link': 'https://www.ncbi.nlm.nih.gov/sra?term=SRX8325541',
   'srr_ids': ['SRR11772360']},
  'GSM4546579': {'submission-date': '2020-05-11',
   'title': 'case1-lung4',
   'accession': 'GSM4546579',
   'organism': 'Homo sapiens',
   'sra_link': 'https://www.ncbi.nlm.nih.gov/sra?term=SRX8325542',
   'srr_ids': ['SRR11772361']},
  'GSM4546580': {'submission-date': '

In [23]:
script_lines = []
for exp_accession, contents in study_dict["experiments"].items():
    if "lung" in contents["title"].lower():
        script_lines.append("echo {}".format(exp_accession))
        this_line = "fasterq-dump -p -o fastqs/{}.fastq {}".format(exp_accession, contents["srr_ids"][0])
        print(this_line)
        script_lines.append(this_line)
        script_lines.append("gzip fastqs/{}*fastq".format(exp_accession))
        script_lines.append("############################")

fasterq-dump -p -o fastqs/GSM4546576.fastq SRR11772358
fasterq-dump -p -o fastqs/GSM4546577.fastq SRR11772359
fasterq-dump -p -o fastqs/GSM4546578.fastq SRR11772360
fasterq-dump -p -o fastqs/GSM4546579.fastq SRR11772361
fasterq-dump -p -o fastqs/GSM4546581.fastq SRR11772363
fasterq-dump -p -o fastqs/GSM4546582.fastq SRR11772364
fasterq-dump -p -o fastqs/GSM4546584.fastq SRR11772366
fasterq-dump -p -o fastqs/GSM4546586.fastq SRR11772368
fasterq-dump -p -o fastqs/GSM4546588.fastq SRR11772370
fasterq-dump -p -o fastqs/GSM4546589.fastq SRR11772371
fasterq-dump -p -o fastqs/GSM4546592.fastq SRR11772374
fasterq-dump -p -o fastqs/GSM4546596.fastq SRR11772378
fasterq-dump -p -o fastqs/GSM4546597.fastq SRR11772379
fasterq-dump -p -o fastqs/GSM4546598.fastq SRR11772380
fasterq-dump -p -o fastqs/GSM4546599.fastq SRR11772381
fasterq-dump -p -o fastqs/GSM4546601.fastq SRR11772383
fasterq-dump -p -o fastqs/GSM4698521.fastq SRR12340058
fasterq-dump -p -o fastqs/GSM4698522.fastq SRR12340059
fasterq-du

In [24]:
with open("download_srrs.sh", "w") as output_stream:
    print("\n".join(script_lines), file = output_stream)

In [25]:
! cat download_srrs.sh

echo GSM4546576
fasterq-dump -p -o fastqs/GSM4546576.fastq SRR11772358
gzip fastqs/GSM4546576*fastq
############################
echo GSM4546577
fasterq-dump -p -o fastqs/GSM4546577.fastq SRR11772359
gzip fastqs/GSM4546577*fastq
############################
echo GSM4546578
fasterq-dump -p -o fastqs/GSM4546578.fastq SRR11772360
gzip fastqs/GSM4546578*fastq
############################
echo GSM4546579
fasterq-dump -p -o fastqs/GSM4546579.fastq SRR11772361
gzip fastqs/GSM4546579*fastq
############################
echo GSM4546581
fasterq-dump -p -o fastqs/GSM4546581.fastq SRR11772363
gzip fastqs/GSM4546581*fastq
############################
echo GSM4546582
fasterq-dump -p -o fastqs/GSM4546582.fastq SRR11772364
gzip fastqs/GSM4546582*fastq
############################
echo GSM4546584
fasterq-dump -p -o fastqs/GSM4546584.fastq SRR11772366
gzip fastqs/GSM4546584*fastq
############################
echo GSM4546586
fasterq-dump -p -o fastqs/GSM4546586.fastq SRR11772368
gzip fastqs/GSM4546586*fas